# Disentangled Multimodal Representation Learning for Recommendation (DMRL)

Ideias:
- tunar os hiperparametros
- paralelizar a previsao
- Adicionar a coluna do assunto em conjunto com o texto
- Limpar os dados de conteudo
- usa a imagem alem do texto
- renomear as colunas para ter um nome mais representativo para o bert
- usar algum algoritmo mais simples apenas para usuarios novos, knn para content based
- ordenar os itens novos que ficaram por ultimo por popularidade 
- Entender o parametro exclude_unknowns=True do RatioSplit e se tem alguma forma do DMRL gerar previsoes para itens e usuarios novos 

Perguntas:


## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import cornac
import numpy as np
import pandas as pd
from cornac.metrics import NDCG
from cornac.eval_methods import RatioSplit
from cornac.data import TextModality
from cornac.models.dmrl.recom_dmrl import DMRL

from utils import load_data, preprocessing_content_data


# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
np.set_printoptions(threshold=np.inf)

/home/matheus/miniconda/envs/sistema_recomendacao_tp2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load and process data

In [3]:
ratings, content, targets = load_data()

In [4]:
ratings["TimestampDate"] = ratings['Timestamp'].dt.date
ratings.loc[ratings.Rating == 0, "Rating"] = 0.01

### Limpar os dados

- FEITO - Year: deletar o - no texto '2013–'
- FEITO - Rated: alterar as diversas formas de escrever NA para NA. Esse e um caso especial
- FEITO - alterar as diversas formas de escrever NA para None em todas as coluans
- FEITO - Language: tem varias linhas que possuem bizarices como  'None, English' e 'None, French' , 'English, None'...
- FEITO - Ratings: criar uma coluna para cada chave do dicionario, entender quais sao todas as chaves que existem

-----------------------------------------------
Variaveis qwue nao precisariam ser tratadas com um bert:
- Metascore
- imdbRating
- Type

In [5]:
content_auxiliar = content.drop(columns=["Poster", "Website", "Response", "Episode", "seriesID"]).copy()

In [6]:
content_auxiliar['Year'] = content_auxiliar['Year'].str.replace('–', '')

In [7]:
nan = content.totalSeasons.unique()[0]
dict_transform_to_na = {
    "Rated":['N/A', 'Not Rated', 'Unrated', 'UNRATED', 'NOT RATED'],
    "all": [nan, 'N/A', 'None', np.nan],
}

for na_value in dict_transform_to_na["all"]:
    content_auxiliar = content_auxiliar.replace(na_value, None)

for na_value in dict_transform_to_na["Rated"]:
    content_auxiliar['Rated'] = content_auxiliar['Rated'].replace(na_value, None)

In [8]:
content_auxiliar['Language'] = content_auxiliar['Language'].str.replace('None, ', '')
content_auxiliar['Language'] = content_auxiliar['Language'].str.replace(', None', '')

In [9]:
# Entendendo os valores possiveis para a coluna Ratings
# A coluna content_auxiliar.Ratings quarda uma lista que posde ter entre 0 e 3 dicionarios. Cada dicionario possui a chave 'Source', 'Value'.
num_ratings_per_item = []
unique_keys = []
rating_sources = []
rating_values = []

for rating_list in content_auxiliar.Ratings:
    num_ratings_per_item.append(len(rating_list))
    for rating_dict in rating_list:
        for key in rating_dict:
            unique_keys.append(key)
        rating_sources.append(rating_dict['Source'])
        rating_values.append(rating_dict['Value'])

set(rating_sources)

{'Internet Movie Database', 'Metacritic', 'Rotten Tomatoes'}

In [10]:
InternetMovieDatabase_list = []
Metacritic_list = []
RottenTomatoes_list = []
for rating_list in content_auxiliar.Ratings:
    InternetMovieDatabase_list.append(None)
    Metacritic_list.append(None)
    RottenTomatoes_list.append(None)
    for rating_dict in rating_list:
        if rating_dict['Source'] == 'Internet Movie Database':
            InternetMovieDatabase_list[-1] = rating_dict['Value']
        elif rating_dict['Source'] == 'Metacritic':
            Metacritic_list[-1] = rating_dict['Value']
        elif rating_dict['Source'] == 'Rotten Tomatoes':
            RottenTomatoes_list[-1] = rating_dict['Value']

In [11]:
content_auxiliar['Internet Movie Database'] = InternetMovieDatabase_list
content_auxiliar['Metacritic'] = Metacritic_list
content_auxiliar['Rotten Tomatoes'] = RottenTomatoes_list


In [12]:
content_auxiliar.drop(columns=['Ratings'], inplace=True)

### Apendar a coluna no valor do dataframe

In [13]:
content_columns = content_auxiliar.columns.to_list()
content_columns.pop(0)

'ItemId'

In [14]:
for column in content_columns:
    content_auxiliar[column] = content_auxiliar[column].apply(lambda x: f"{column}: {x}; " if x is not None else f"{column}: unknown value; ")

In [15]:
content_processed = content_auxiliar[['ItemId']].copy()
content_processed["text"] = content_auxiliar[content_columns].astype(str).fillna('').agg(' '.join, axis=1)

## Basic analysis

In [ ]:
ratings.head()

In [ ]:
# Number of unique users and items
ratings.UserId.nunique(), ratings.ItemId.nunique()

In [ ]:
# how many itens purchased by each user purchase
ratings.groupby(["UserId", 'Timestamp'])["ItemId"].nunique().value_counts()

In [ ]:
# how many itens purchased by each user day by day
ratings.groupby(["UserId", 'TimestampDate'])["ItemId"].nunique().value_counts()

In [ ]:
# how many times each user purchased items
ratings.groupby("UserId")['Timestamp'].nunique().value_counts()

In [ ]:
# how many times each user purchased items per day
ratings.groupby("UserId")['TimestampDate'].nunique().value_counts()

In [ ]:
content.isna().sum()

In [ ]:
ratings.Rating.unique()

## Train model

In [16]:
item_text_modality = TextModality(
    corpus=content_processed.text.to_list(),
    ids=content_processed.ItemId.to_list(),
)

In [18]:
ratio_split = RatioSplit(
    data=ratings[['UserId', 'ItemId', 'Rating']].values.tolist(),
    test_size=0.2,
    exclude_unknowns=True,
    verbose=True,
    seed=12012001,
    rating_threshold=0.5,
    item_text=item_text_modality,
)

rating_threshold = 0.5
exclude_unknowns = True
---
Training data:
Number of users = 46750
Number of items = 27045
Number of ratings = 527776
Max rating = 10.0
Min rating = 0.0
Global mean = 7.3
---
Test data:
Number of users = 46750
Number of items = 27045
Number of ratings = 124031
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 46750
Total items = 27045


In [19]:
# Instantiate DMRL recommender
dmrl_recommender = DMRL(
    batch_size=4096,
    epochs=20,
    log_metrics=False,
    learning_rate=0.01,
    num_factors=2,
    decay_r=0.5,
    decay_c=0.01,
    num_neg=3,
    embedding_dim=100,
)

In [20]:
# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=ratio_split, models=[dmrl_recommender], metrics=[NDCG()]
).run()


[DMRL] Training started!


/home/matheus/miniconda/envs/sistema_recomendacao_tp2/lib/python3.9/site-packages/cornac/models/dmrl/transformer_text.py:67: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sa

Pre-encoding the entire corpus. This might take a while.
Using device cuda for training
  batch 5 loss: 2839.728076171875
  batch 10 loss: 2839.757373046875
  batch 15 loss: 2831.77578125
  batch 20 loss: 2791.44833984375
  batch 25 loss: 2686.66455078125
  batch 30 loss: 2593.422119140625
  batch 35 loss: 2508.665576171875
  batch 40 loss: 2348.50712890625
  batch 45 loss: 2184.107275390625
  batch 50 loss: 1982.2599853515626
  batch 55 loss: 1783.6734619140625
  batch 60 loss: 1682.8436767578125
  batch 65 loss: 1657.6788330078125
  batch 70 loss: 1630.51259765625
  batch 75 loss: 1584.9889404296875
  batch 80 loss: 1565.05732421875
  batch 85 loss: 1569.6202880859375
  batch 90 loss: 1541.485791015625
  batch 95 loss: 1539.0696533203125
  batch 100 loss: 1553.11025390625
  batch 105 loss: 1523.523193359375
  batch 110 loss: 1503.656591796875
  batch 115 loss: 1537.691845703125
  batch 120 loss: 1498.9853515625
  batch 125 loss: 1510.45986328125
Epoch: 0 is done
  batch 5 loss: 1361.

Ranking: 100%|██████████| 19975/19975 [05:23<00:00, 61.71it/s]


TEST:
...
     | NDCG@-1 | Train (s) | Test (s)
---- + ------- + --------- + --------
DMRL |  0.2387 |  126.9726 | 323.7108



In [73]:
# como usar esse algoritmo para tratar itens novos?
# # como usar esse algoritmo para tratar usuarios novos?


# eu tenho o conteudo de todos os itens, incluive itens que estao apenas n conjunto de teste?

In [21]:
target_prediction = targets.copy()
target_prediction["Rating"] = -1

user_id_list = targets.UserId.unique()
for user_id in user_id_list:
    # Get the train dataframe index of the user to predict
    user_index = ratio_split.train_set.uid_map.get(user_id)

    if user_index is None:
        print(f"User {user_id} is not in the train set")
        continue

    # Flter by items to predict 
    items_to_predict = targets.loc[targets.UserId == user_id, "ItemId"].to_list()

    # Get the train dataframe index of the items to predict
    items_to_predict_index = np.array([ratio_split.train_set.iid_map.get(item_id) for item_id in items_to_predict])

    items_to_predict_tensor = torch.tensor([idx for idx in items_to_predict_index if idx is not None])

    # Get the position of items that are not in the train set
    none_indices = [i for i, x in enumerate(items_to_predict_index) if x is None]

    # Get the prediction for the items
    line_rating = dmrl_recommender.score(user_index=user_index, item_indices=items_to_predict_tensor)

    # Insert -1 in the position of items that are not in the train set
    for index_to_insert in none_indices:
        line_rating = np.insert(line_rating, index_to_insert, -1)

    # Insert the prediction in the target_prediction dataframe
    target_prediction.loc[targets.UserId == user_id, "Rating"] = line_rating

User 02b780c583 is not in the train set
User 03fbe3c1e8 is not in the train set
User 0503bf6097 is not in the train set
User 0958560bd4 is not in the train set
User 0b17c3df40 is not in the train set
User 10c3aef33f is not in the train set
User 119653373a is not in the train set
User 139ccdbf5a is not in the train set
User 1671fa4df3 is not in the train set
User 16757f4d90 is not in the train set
User 187454feb4 is not in the train set
User 1af811e7b9 is not in the train set
User 1bbf59e4a4 is not in the train set
User 1ef563bb8b is not in the train set
User 1f6a565547 is not in the train set
User 211a7a84d3 is not in the train set
User 243be2818a is not in the train set
User 24a13b72fc is not in the train set
User 27649dc46e is not in the train set
User 2a3500f75b is not in the train set
User 2ae6906d5d is not in the train set
User 2cfdb14e05 is not in the train set
User 2ed4d0c23c is not in the train set
User 2ef13886ae is not in the train set
User 302c6ebc7d is not in the train set


In [22]:
target_prediction = target_prediction.sort_values(["UserId", "Rating"], ascending=[True, False])

In [23]:
target_prediction.to_csv("submissao_3_DMRL_versao_2.csv", index=False)

In [24]:
target_prediction = target_prediction.drop(columns="Rating")

In [25]:
target_prediction.to_csv("submissao_3_DMRL_versao_2_sem_rating.csv", index=False)